In [9]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# POC SPEECH ANALITYCS
---

Prova de conceito do mecanismo de extração de insights a partir de áudios. A partir de um arquivo de áudio, o motor desenvolvido extrai o conteúdo (texto), trata o resultado e analisa o teor sentimental disposto na sentença.

## Estudo de caso 
Análise sentimental de duas posições políticas (PT e Democratas) sobre a decisão do STJ sobre o ex-presidente Lula. 
Data: 06 de março de 2018 / 
Fonte: [Rádioagencia Nacional](http://radioagencianacional.ebc.com.br/geral/audio/2018-03/decisao-do-stj-sobre-o-ex-presidente-lula-repercute-na-camara-dos-deputados?editoria_id=All&page=1)

In [3]:
import io
import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Volumes/Toshiba/Cloud/pCloud Drive/Profissional/Zurve/SpeechAnalytics/ds-linx-a60766b872ae.json"

# Google Cloud client library - SPEECH
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
gcSpeech = speech.SpeechClient()

# Google Cloud client library - NLP
from google.cloud import language
from google.cloud.language import enums as enumsNLP
from google.cloud.language import types as typesNLP
gcLanguage = language.LanguageServiceClient()

In [4]:
def zSpeech2Text(audio_flac_file):

    # Carregar audio na memoria
    with io.open(audio_flac_file, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)
    # Ajustar configuracoes do Speech2Text
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=44100,
        language_code='pt-BR')

    # Detectar conteúdo do audio
    response = gcSpeech.recognize(config, audio)
    
    return response

def zSentimental(text):
    document = typesNLP.Document(
        content=text,
        type=enumsNLP.Document.Type.PLAIN_TEXT)

    # Detectando sentimento presente no texto
    sentiment = gcLanguage.analyze_sentiment(document=document).document_sentiment
    
    # Interpretar indicadores: https://cloud.google.com/natural-language/docs/basics#interpreting_sentiment_analysis_values
    # Score:  emoção de modo geral (-1 Negativo / 1 Positivo)
    # Magnitude: intensidade geral da emoção, positiva e negativa, no texto fornecido (0.0 a +inf.) 
    #   Ao contrário de score, magnitude não é normalizada. Cada expressão de emoção no texto, positiva e 
    #   negativa, contribui para a magnitude do texto. Por isso, blocos de texto mais longos podem ter 
    #   magnitudes maiores.
    if sentiment.score < -0.5:
        profile = 'Negativo'
    elif sentiment.score > 0.5:
        profile = 'Positivo'  
    else:
        profile = 'Neutro'
        
    if (sentiment.magnitude > 2) and (profile == 'Positivo' or profile == 'Negativo'):
        profile = 'Claramente' + profile
    
    sentimento = {'score': sentiment.score, 'magnitude': sentiment.magnitude, 'profile': profile}
    return sentimento

def analysis(conteudo):
    texto = conteudo.alternatives[0].transcript
    prob = round(conteudo.alternatives[0].confidence * 100, 2)
    sentimento = zSentimental(texto)
    print('=============== SPEECH ANALYTICS ===============')
    print('Texto: "(...) {}."'.format(texto))
    print('-')
    print('Tx. Prob.: {}%'.format( prob ))
    print('================================================')
    print('============= SENTIMENT ANALYSIS ===============')
    print('Score: {}."'.format(sentimento['score']))
    print('Magnitude: {}."'.format(sentimento['magnitude']))
    print('-')
    print('Resultado: {}'.format(sentimento['profile']))
    print('================================================')
    

### Opinião 1: Paulo Pimenta - PT (líder do partido na Câmara dos Deputados)

In [5]:
# Extraindo texto do audio
conteudo = zSpeech2Text('opiniao1.flac')

In [6]:
# Interpretando o resultado
if len(conteudo.results) > 0:
    analysis(conteudo.results[0])
else:
    print('Não foi possível extrair o conteúdo do áudio.\nSelecione outro áudio e tente novamente.')

=============== SPEECH ANALYTICS ===============
Texto: "(...) com a decisão o tribunal não entrou da análise constitucional ele entendeu diante da decisão do supremo tribunal federal Não restava ao STJ contra decisão diferente do que esta que foi adotada hoje Portanto mais do que nunca torna-se imperioso para o respeito à ampla defesa de qualquer cidadão e não particularmente somente do presidente Lula que ele terá direito de ter o seu abraço corpos analisado pelo pleno do Supremo Tribunal."
-
Tx. Prob.: 94.05%
============= SENTIMENT ANALYSIS ===============
Score: -0.20000000298023224."
Magnitude: 0.20000000298023224."
-
Resultado: Neutro


### Opinião 2: Rodrigo Garcia - Democratas (líder do partido na Câmara dos Deputados)

In [7]:
# Extraindo texto do audio
conteudo = zSpeech2Text('opiniao2.flac')

In [8]:
# Interpretando o resultado
if len(conteudo.results) > 0:
    analysis(conteudo.results[0])
else:
    print('Não foi possível extrair o conteúdo do áudio.\nSelecione outro áudio e tente novamente.')

=============== SPEECH ANALYTICS ===============
Texto: "(...) correta a decisão hoje ela foi acertada respeita a lei Supremo Tribunal Federal tem o entendimento em vigor da prisão em Segunda instância e o que fez o STJ foi confirmar esse entendimento mostra que as instituições funcionam que o poder judiciário aplica a lei que está em vigência que ninguém."
-
Tx. Prob.: 93.35%
============= SENTIMENT ANALYSIS ===============
Score: 0.6000000238418579."
Magnitude: 0.6000000238418579."
-
Resultado: Positivo


In [10]:
#zSentimental('Nem a pau que isso vai dar certo. Mas talvez, seja bom')

In [ ]:
# Prep audio
# ffmpeg -i test.mp3 -ac 1 test.flac
# ffmpeg -ss 0 -t 30 -i portugues.mp3 -ac 1 portugues30.flac